In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0c5dbcce7022be5b24f3e9fcc70805a80bc1dc983d0534daf555bac5038a99b4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import accuracy_score

# Reading the Dataset

In [3]:
spark = SparkSession.builder.appName('musicstreaming').getOrCreate()
df = spark.read.csv('music_streaming.csv', header = True, inferSchema = True)

# data cleaning & engineering

In [4]:
df.printSchema()

root
 |-- Artist Name: string (nullable = true)
 |-- Track Name: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_in min/ms: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- Genre: double (nullable = true)



In [5]:
df.show()

+--------------------+--------------------+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+--------------+-----+
|         Artist Name|          Track Name|Popularity|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_in min/ms|time_signature|Genre|
+--------------------+--------------------+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+--------------+-----+
|          Bruno Mars|That's What I Lik...|        60|       0.854| 0.564|  1|  -4.964| 1.0|     0.0485|      0.0171|            NULL|  0.0849|  0.899|134.071|          234596.0|           4.0|  5.0|
|              Boston|        Hitch a Ride|        54|       0.382| 0.814|  3|   -7.23| 1.0|     0.0406|      0.0011|         0.00401|   0.101|  0.569|116.454|          251733.0|           4.0| 10.0|


In [6]:
#drop missing values
df = df.na.drop()

In [7]:
#drop duplicates
df = df.dropDuplicates()

In [8]:
df.dtypes

[('Artist Name', 'string'),
 ('Track Name', 'string'),
 ('Popularity', 'string'),
 ('danceability', 'string'),
 ('energy', 'string'),
 ('key', 'int'),
 ('loudness', 'double'),
 ('mode', 'double'),
 ('speechiness', 'double'),
 ('acousticness', 'double'),
 ('instrumentalness', 'double'),
 ('liveness', 'double'),
 ('valence', 'double'),
 ('tempo', 'double'),
 ('duration_in min/ms', 'double'),
 ('time_signature', 'double'),
 ('Genre', 'double')]

# Queries (SparkSQL)


In [9]:
# Register DataFrame as a table for Spark SQL
df.createOrReplaceTempView("songs")

In [10]:
# a) Which genre has the highest average popularity?
query_a_sql = """
SELECT Genre, AVG(CAST(Popularity AS DOUBLE)) AS avg_popularity
FROM songs
GROUP BY Genre
ORDER BY avg_popularity DESC
LIMIT 1
"""
result_a_sql = spark.sql(query_a_sql)
result_a_sql.show()

+-----+------------------+
|Genre|    avg_popularity|
+-----+------------------+
|  4.0|55.140845070422536|
+-----+------------------+



In [11]:
# b) Display which artist has recorded the most number of songs with a duration of more than 5 minutes (Spark SQL)
query_b_sql = """
SELECT `Artist Name`, COUNT(*) AS num_songs
FROM songs
WHERE `duration_in min/ms` > 5
GROUP BY `Artist Name`
ORDER BY num_songs DESC
LIMIT 1
"""

result_b_sql = spark.sql(query_b_sql)
result_b_sql.show()

+------------------+---------+
|       Artist Name|num_songs|
+------------------+---------+
|The Rolling Stones|       26|
+------------------+---------+



In [12]:
# c) How many songs are included in every Genre? (Spark SQL)
query_c_sql = """
SELECT Genre, COUNT(*) AS num_songs
FROM songs
GROUP BY Genre
ORDER BY num_songs DESC
"""

result_c_sql = spark.sql(query_c_sql)
result_c_sql.show()

+-------+---------+
|  Genre|num_songs|
+-------+---------+
|   10.0|     2919|
|    6.0|     1783|
|    8.0|     1397|
|    1.0|      958|
|    2.0|      892|
|    9.0|      845|
|    5.0|      420|
|    7.0|      372|
|    0.0|      362|
|    3.0|      247|
|    4.0|      142|
|117.017|        1|
+-------+---------+



In [13]:
# d) Which artists dominated the charts? (Spark SQL)
query_d_sql = """
SELECT `Artist Name`, COUNT(*) AS num_songs
FROM songs
GROUP BY `Artist Name`
ORDER BY num_songs DESC
LIMIT 5
"""

result_d_sql = spark.sql(query_d_sql)
result_d_sql.show()

+------------------+---------+
|       Artist Name|num_songs|
+------------------+---------+
|The Rolling Stones|       28|
|    Britney Spears|       27|
|                U2|       25|
|         Metallica|       24|
|      Led Zeppelin|       22|
+------------------+---------+



In [14]:
# e) Recommend at least 5 fun/not-boring songs that can be played at a party (Spark SQL)
query_e_sql = """
SELECT `Artist Name`, `Track Name`, energy, danceability, valence
FROM songs
ORDER BY (energy + danceability + valence) DESC
LIMIT 5
"""

result_e_sql = spark.sql(query_e_sql)
result_e_sql.show()

+---------------+--------------------+------+------------+-------+
|    Artist Name|          Track Name|energy|danceability|valence|
+---------------+--------------------+------+------------+-------+
|LCD Soundsystem|Daft Punk Is Play...| 0.856|       0.921|  0.976|
|  Isaiah Rashad|Lay Wit Ya (feat....| 0.859|       0.925|  0.967|
|    Killer Mike|          Kryptonite| 0.916|       0.863|  0.964|
|        Shampoo|             Trouble| 0.971|       0.796|  0.963|
|   Prince Rapid|Gladiator - Instr...| 0.972|        0.78|  0.958|
+---------------+--------------------+------+------------+-------+

